In [ ]:
import pandas as pd
import numpy as np
import random
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import GEOparse
import os
import math
import sklearn.cluster

In [ ]:
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (15,10)

## preparing the data - reading the dataframe, normalizing the values and cleaning NAs

In [ ]:
raw_data = GEOparse.get_GEO(filepath = "GDS2771.soft" , silent = True) 

read gene expression data (file should be save in the same directory as the notebook).
Analysis of large airway epithelial cells from cigarette smokers without cancer, 
with cancer, and with suspect lung cancer. 
We would like to try and use the gene expression values to build a model that
predicts if a smoker has lung cancer or not.

In [ ]:
all_data = raw_data.table.set_index('ID_REF')
all_data.shape

Data has 193 patients and 22283 gene expression values per patient.  

In [ ]:
data = all_data.copy().dropna()
cancer_stat = raw_data.columns['disease state']
identifier = data['IDENTIFIER']
data = data.drop('IDENTIFIER', axis=1)

In [ ]:
data = pd.DataFrame(normalize(data), index = data.index, columns = data.columns ) # normalize the gene expression values before the prediction

In [ ]:
data.shape # size of the data (gene expression values * patients) after clean up

# statistics
We would like to see if in general, there are genes that are differentially expressed between patient with and without cancer. To do so, we will run a t-test with the gene expression values between the two groups (cancer / no-cancer).
We will store the p_values and the coefficients (for over and under expression in the cancer group) in a dataframe.

In [ ]:
results = pd.DataFrame(index = data.index,columns = ["t_test_cancer_overexpressed","t_test_cancer_underexpressed"])
cancer_data = data[[data.columns[i] for i in range(192) if cancer_stat[i]=="cancer"]]
nocancer_data = data[[data.columns[i] for i in range(192) if cancer_stat[i]=="no cancer"]]
for i in data.index:
    t_coef,t_pval = scipy.stats.ttest_ind(cancer_data.loc[i],nocancer_data.loc[i])
    t_pval = t_pval/2
    if t_coef<0:
        t_pval = 1-t_pval
    results.at[i,"t_test_cancer_overexpressed"] = t_pval
    results.at[i,"t_test_cancer_underexpressed"] = 1-t_pval

In [ ]:
t_over = (results['t_test_cancer_overexpressed'] < 0.05).sum()
t_under = (results['t_test_cancer_underexpressed'] < 0.05).sum()
print('t_test overexpressed: {} \nt_test underexpressed: {}'.format(t_over, t_under))

We found that 3831 genes are overexpressed in the cancer group, and 3335 genes are underexpressed in the cancer group.
Before correction for multiple testing.

In [ ]:
fig,ax = plt.subplots()
sns.lineplot(x = np.sort(np.random.random(data.shape[0])), y = [i for i in range(data.shape[0])],ax=ax,label = "expected p_values")
sns.lineplot(x = results["t_test_cancer_overexpressed"].sort_values(),y = [i for i in range(data.shape[0])],ax=ax,label = "t_test p_values")
ax.set_title("cancer overexpressed - expected vs. observed p values")

We can see that the p-values we got (blue) are different than what we would get if there would be no difference in gene expression profiles between patiens with and without cancer.
Looks promising for our future classifier! 

In [ ]:
t_fdr = statsmodels.stats.multitest.multipletests(results["t_test_cancer_overexpressed"], method='fdr_bh')[1]
for i in [0.1,0.05,0.01,0.001]:
    print ("T_test significant at FDR = {}: {} genes.".format(str(i),str((t_fdr<i).sum())))

See above # significant tests <b>after correcting for multiple testing</b>. Still many significant tests! meaning that the difference between the groups is not random!

Next, we want to ask if some genes correlate (are expressed together) within the cancer class. We will compute correlations between each pair, store the p-values, and correct for multiple testing.
We will do it for the 50 most over/under expressed genes in the cancer group.

In [ ]:
D = list(results.sort_values("t_test_cancer_overexpressed").index[0:50]) +list(results.sort_values("t_test_cancer_underexpressed").index[0:50])

In [ ]:
corrs= pd.DataFrame(index = D,columns = D)
pval = pd.DataFrame(index = D,columns = D)
for gene1 in D:
    for gene2 in D:
        if (gene1!=gene2) and (np.isnan(corrs.at[gene1,gene2])) and (np.isnan(corrs.at[gene2,gene1])):
            corrs.at[gene1,gene2], pval.at[gene1,gene2] = scipy.stats.spearmanr(cancer_data.loc[gene1],cancer_data.loc[gene2])
corrected_pval = statsmodels.stats.multitest.multipletests(pval.stack(),method="fdr_bh")[1]

In [ ]:
print('2) {} gene pairs are co-expressed within the cancer class!'.format((corrected_pval <= 0.05).sum().sum()))

## cross validation for number of over and under expressed genes

Next, we want to build a classifier and preform cross validation - to test if different number of over and under regulated genes that we use in the model can affect the results. To do so, we will test the accuracy parameters when taking any number from 1 to 25 of the top overexpressed and underexpressed genes for our GaussianNB classifier.
Ideally, we would like to take a number of genes that provides as much information as possible, <b>while avoiding over fitting</b>

In [ ]:
final = pd.DataFrame(index = range(1,26), columns = ["tp","tn","fp","fn","accuracy"])
kf = KFold(n_splits=10,random_state=7,shuffle=True)
for i in range(1,26):
    indexes = list(results.sort_values("t_test_cancer_overexpressed").index[0:i]) + list(results.sort_values("t_test_cancer_underexpressed").index[0:i])
    accuracies = []
    tps = []
    tns = []
    fps = []
    fns = []
    for train_index, test_index in kf.split(data.T[cancer_stat!="suspect cancer"]):
        X_train, X_test = data.loc[indexes].T[cancer_stat!="suspect cancer"].iloc[train_index], data.loc[indexes].T[cancer_stat!="suspect cancer"].iloc[test_index]
        y_train, y_test = cancer_stat[cancer_stat!="suspect cancer"][train_index], cancer_stat[cancer_stat!="suspect cancer"][test_index]
        model = GaussianNB()
        model.fit(X_train,y_train)
        preds = model.predict(X_test)
        con = pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)),index = ["cancer","no cancer"], columns = ["cancer","no cancer"])
        con.index.name = "class"
        con.columns.name = "preds"
        tps.append(con['cancer'][0]/(con['no cancer'][0]+con['cancer'][0]))
        tns.append(con['no cancer'][1]/(con['no cancer'][1]+con['cancer'][1]))
        fps.append(con['cancer'][1]/(con['no cancer'][1]+con['cancer'][1]))
        fns.append(con['no cancer'][0]/(con['no cancer'][0]+con['cancer'][0]))
        accuracies.append((con['cancer'][0]+con['no cancer'][1])/(con.sum()[0:2].sum()))
    final.at[i,"accuracy"] = np.array(accuracies).mean()
    final.at[i,"tp"] = np.array(tps).mean()
    final.at[i,"tn"] = np.array(tns).mean()
    final.at[i,"fp"] = np.array(fps).mean()
    final.at[i,"fn"] = np.array(fns).mean()       

In [ ]:
final

In [ ]:
fig,ax = plt.subplots()

final.plot.line(ax=ax)
ax.set_title("CV - accuracy parameters as a function of over and under expressed gene # for classifier")
ax.set_xlabel("number of under and over regulated genes (each direction)")

As we can see, our model improves as we add genes (features) up until 12 under expressed and 12 overexpressed genes. Adding additional features after 12 genes doesn't improve the model (maybe even causes a slight overfit). This means that after selecting the 24 most regulated genes, there is no additional information that is unique to the additional genes we could add to the model.

See below boxplots of the 12 most overexpressed genes in the cancer group - just as a nice visualization of the difference in expression between patients with and without cancer.

In [ ]:
fig,ax = plt.subplots()
top_12 = results.sort_values("t_test_cancer_overexpressed").index[0:12]
plot_top12 = data.T[top_12]
plot_top12["classes"] = cancer_stat[cancer_stat != 'suspect cancer']
sns.boxplot(data=pd.melt(plot_top12,"classes"),x="ID_REF",y="value",hue="classes")
ax.set_title("cancer overexpression, top 12 genes")
ax.set_ylabel("expression")

In [ ]:
most_genes = list(results.sort_values("t_test_cancer_overexpressed").index[0:12]) + list(results.sort_values("t_test_cancer_underexpressed").index[0:12])
X_train, X_test, y_train, y_test = train_test_split(data.loc[most_genes].T[cancer_stat!="suspect cancer"], cancer_stat[cancer_stat!="suspect cancer"], test_size=0.2,random_state=7)
model = GaussianNB()
model.fit(X_train,y_train)
preds = model.predict(X_test)

In [ ]:
con = pd.DataFrame(confusion_matrix(y_train,model.predict(X_train)),index = ["cancer","no cancer"], columns = ["cancer","no cancer"])
con.index.name = "class"
con.columns.name = "preds"

print("confusion matrix - training set")

con

In [ ]:
tp = con['cancer'][0]/(con['no cancer'][0]+con['cancer'][0])
tn = con['no cancer'][1]/(con['no cancer'][1]+con['cancer'][1])
fp = con['cancer'][1]/(con['no cancer'][1]+con['cancer'][1])
fn = con['no cancer'][0]/(con['no cancer'][0]+con['cancer'][0])
accuracy = (con['cancer'][0]+con['no cancer'][1])/(con.sum()[0:2].sum())
print('True Positive Rate: {} \nTrue Negative Rate: {} \nFalse Positive Rate: {} \n\
False Negative Rate: {} \nAccuracy: {}'.format(tp, tn, fp, fn, accuracy))

In [ ]:
con = pd.DataFrame(confusion_matrix(y_test,model.predict(X_test)),index = ["cancer","no cancer"], columns = ["cancer","no cancer"])
con.index.name = "class"
con.columns.name = "preds"

print("confusion matrix - testing set")

con

In [ ]:
tp = con['cancer'][0]/(con['no cancer'][0]+con['cancer'][0])
tn = con['no cancer'][1]/(con['no cancer'][1]+con['cancer'][1])
fp = con['cancer'][1]/(con['no cancer'][1]+con['cancer'][1])
fn = con['no cancer'][0]/(con['no cancer'][0]+con['cancer'][0])
accuracy = (con['cancer'][0]+con['no cancer'][1])/(con.sum()[0:2].sum())
print('True Positive Rate: {} \nTrue Negative Rate: {} \nFalse Positive Rate: {} \n\
False Negative Rate: {} \nAccuracy: {}'.format(tp, tn, fp, fn, accuracy))

# Not a bad classifier!!!

## k-means and PCA

lastly, we want to visually cluster the patients based on their gene expression values for the 12 most over and 12 most under expressed genes in the cancer group. We are wondering if we can run a PCA analysis and see that the groups cluster separatly based on a k-mean analysis.

In [ ]:
indexes = list(results.sort_values("t_test_cancer_overexpressed").index[0:12]) + list(results.sort_values("t_test_cancer_underexpressed").index[0:12])
k_data = data.loc[indexes].T[cancer_stat!="suspect cancer"]

In [ ]:
kmeans = sklearn.cluster.KMeans(2,random_state = 7)

In [ ]:
clusters = kmeans.fit_predict(k_data)

In [ ]:
clusters = ["cancer" if i==0 else "no cancer" for i in clusters]

In [ ]:
pca = sklearn.decomposition.PCA()

In [ ]:
pcs = pca.fit(k_data)
explained = pcs.explained_variance_ratio_
pcs = pcs.transform(k_data)

In [ ]:
plot = pd.DataFrame(index = k_data.index, columns = ["PC1","PC2","cancer","cluster"])
plot["PC1"] = pcs[:,1]
plot["PC2"] = pcs[:,2]
plot["cancer"] = cancer_stat[cancer_stat!="suspect cancer"]
plot["cluster"] = clusters
plot["correct"] = plot["cancer"] == plot["cluster"]
fig,ax = plt.subplots()
sns.scatterplot(data = plot, x = "PC1", y = "PC2", hue = "cancer" , style = "cluster", markers = [",","o"] ,ax=ax)
ax.set_xlabel("PC1 {}%".format(str(round(explained[0]*100,2))))
ax.set_ylabel("PC2 {}%".format(str(round(explained[1]*100,2))))
ax.set_title("PCA for gene expression - cancer state and clusters")

##### See above the patients plotted on 2 dimentions, based on their gene expression values. In Colors we can see if the patient has cancer, or if he/she is healthy. in the shape we can see the prediction, based on kmeans

In [ ]:
fig,ax = plt.subplots()
sns.scatterplot(data = plot, x = "PC1", y = "PC2", style = "correct" , color = "black",markers = ["X","o"],ax=ax)
ax.set_xlabel("PC1 {}%".format(str(round(explained[0]*100,2))))
ax.set_ylabel("PC2 {}%".format(str(round(explained[1]*100,2))))
ax.set_title("PCA for gene expression - {}/{} correct clustering".format(str(plot["correct"].sum()),str(plot.shape[0])))

###### Using the PCA and k-means approach, we predicted 129 pateints correctly. Overall, the GaussianNB classifier behaved better.